Bigram model following Andrej Karpathy's GPT tutorial on Youtube (part 1 of 2). 

Trained using the full works of H.P. Lovecraft instead of Shakespeare. Unfortunately the txt I found has some artifacts. 

In [60]:
#imports

import torch
import torch.nn as nn
from torch.nn import functional as F

#hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------



In [13]:
#Processing the input data
with open('lovecraft.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)


['\n', '\x0c', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '°', '·', '¿', 'Å', 'Æ', '×', 'á', 'ä', 'æ', 'è', 'é', 'ê', 'ë', 'ï', 'ñ', 'ö', 'ü', 'Ν', 'Ο', 'Π', 'Σ', 'Υ', '–', '—', '―', '‖', '‗', '‘', '’', '”', '•']


In [14]:
#Encoding and decoding functions from strings to list of numbers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] #String to list
decode = lambda l: "".join(itos[i] for i in l) #List to string


[35, 60, 67, 67, 70, 2, 78, 70, 73, 67, 59]
Hello world!


In [19]:
#Wrapping data into a torch tensor
data = torch.tensor(encode(text), dtype = torch.long)

#Training/validation split; we will use a 95/5 ratio
n = int(len(data) * 0.95)
train_data = data[:n]
val_data = data[n:]



In [39]:
#Split data into batches and blocks to train 

def get_batch(split):
    data = train_data if split == 'train' else val_data 
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix]) #context array
    y = torch.stack([data[i+1:i + block_size + 1] for i in ix]) #target array
    return x,y

In [25]:
xb,yb = get_batch('train')

#Sample loader for training data
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print("context:", context)
        print("target: ", target)


context: tensor([76])
target:  tensor(71)
context: tensor([76, 71])
target:  tensor(2)
context: tensor([76, 71,  2])
target:  tensor(63)
context: tensor([76, 71,  2, 63])
target:  tensor(64)
context: tensor([76, 71,  2, 63, 64])
target:  tensor(74)
context: tensor([76, 71,  2, 63, 64, 74])
target:  tensor(2)
context: tensor([76, 71,  2, 63, 64, 74,  2])
target:  tensor(74)
context: tensor([76, 71,  2, 63, 64, 74,  2, 74])
target:  tensor(67)
context: tensor([60])
target:  tensor(71)
context: tensor([60, 71])
target:  tensor(76)
context: tensor([60, 71, 76])
target:  tensor(75)
context: tensor([60, 71, 76, 75])
target:  tensor(60)
context: tensor([60, 71, 76, 75, 60])
target:  tensor(59)
context: tensor([60, 71, 76, 75, 60, 59])
target:  tensor(2)
context: tensor([60, 71, 76, 75, 60, 59,  2])
target:  tensor(75)
context: tensor([60, 71, 76, 75, 60, 59,  2, 75])
target:  tensor(70)
context: tensor([57])
target:  tensor(67)
context: tensor([57, 67])
target:  tensor(70)
context: tensor([57

In [61]:
#Bigram model: using the only the previous character to determine output

class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are (B,T) tensors of integers
        # logits are (B,T,C) tensors; last coordinate denotes the probability distribution of the next token
        
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) #Get predictions
            logits = logits[:,-1,:] #Take logits for the last time step; (B,C) tensor
            probs = F.softmax(logits, dim=-1) #Probabilities for the next token
            idx_next = torch.multinomial(probs,num_samples=1) #(B,1) tensor after sampling the next token
            idx = torch.cat((idx,idx_next), dim = 1) #Concatenate new token into running sequence, (B,T+1) tensor
        return idx

    
m = BigramLanguageModel()
logits, loss = m(xb, yb)


In [53]:
#Sample text generation

idx = torch.zeros((1,1), dtype=torch.long)
new_text = m.generate(idx, max_new_tokens=500)[0].tolist()
print(decode(new_text))


we tpty ts gaw
cof
silli‘site Y#YEng iale, D‘s thandichinunts se angherofole, Th.
/R&xtle tar t‘s w t,
k orin. tstl
toorreass.
d at. f can. anovicollesin athalycas
Alde trsk hefld l st sin win bon needenounereaches tald, H. s, orori, I thaiad buife ggnssphend w. Care ve som heegs, a ant whonssilirk ticergn f rerr festat urdeas s wn Lisqulasureacho ―Fothay g ce imofore d je ff Thedvomabe s ce Ba t fous by ghamesind or, aesinishereczichite.
halereron thandencalirinirnvecimill d hinchile Consued 


In [51]:
#Gradient descent with PyTorch Adam optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

training_steps = 1000
for steps in range(training_steps):
    #Get training data
    xb, yb = get_batch('train')

    #Evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


2.5186614990234375
